In [60]:
import math
import numpy as np

In [61]:
# read bed file
def read_bedfile(bedfile):
    '''
    :param bedfile:  name of bed file  
    :return ls:   list in the bed file
    '''
    f=open(bedfile,"r")#"./SC-2F-N503702-18C_40000_abs.bed"
    ls=[]
    for line in f:
        x=line.strip('\n').split('\t')
        ls.append(x)
        
    f.close()
    return ls

In [62]:
# read contact file of hicpro output 
def read_contactfile(matrixfile):
    '''
    :param matrixfile:  name of hicpro output  
    :return ls:   contact information list
    '''
    f=open(matrixfile,"r")#"./SC-2F-N503702-18C_40000.matrix"
    ls=[]
    for line in f:
        x=line.strip('\n').split('\t')
        x[2]=str(float(x[2]))
        ls.append(x)
        
    f.close()
    return ls

In [64]:
#read chromosome size file  
def get_chrom_size_dict(filename):
    '''
    :param matrixfile:  name of chromosome size file   
    :return d:   chromosome size dict
    '''
    f=open(filename,"r")
    ls=[]
    for line in f:
        ls.append(line.strip('\n').split('\t'))
    f.close()
    d=dict()
    for line in ls:
        d[line[0]] = line[1]
    return d

In [77]:
# get contact information per chromosome 
def get_chr_contact(chr_name,contact_list,bin_index_list):
    '''
    :param chr_name:  name of chromosome
    :param contact_list:  name of contact matrix file 
    :param bin_index_list:  the interaction start and end coordinates for each chromosome in the contact matrix file     
    :return chr_contact_list:   chromosome contact list
    '''    
    chr_contact_list=[]
    if chr_name=="X":
        index=18
    elif chr_name=="Y":
        index=19
    elif chr_name=="MT":
        index=20
    else:
        index=eval(chr_name)-1
        
    start=bin_index_list[index]
    end=bin_index_list[index+1]
    for item in contact_list:
        if eval(item[0])>=start and eval(item[0])<end and eval(item[1])>=start and eval(item[1])<end :
            chr_contact_list.append(item)
    return chr_contact_list

In [67]:
# get contact matrix that suits for deDoc2
def get_contact_matrix(chr_contact_list,chr_name,binsize):
    '''
    :param chr_contact_list:  chromosome contact list
    :param chr_name:  name of chromosome
    :param binsize:  resolution
    :return contact_matrix:   contact matrix that suits for deDoc2
    '''       
    chr_size_dict=get_chrom_size_dict('./Sus_scrofa.chrom.sizes')
    chr_size=chr_size_dict[chr_name]
    bin_num=math.ceil(eval(chr_size)/eval(binsize))
    contact_matrix=np.zeros((bin_num,bin_num), dtype=int)

    for line in chr_contact_list:
        x_bin=eval(line[0])
        y_bin=eval(line[1])
        c=eval(line[2])
        contact_matrix[x_bin-1,y_bin-1]+=c 
        contact_matrix[y_bin-1,x_bin-1]+=c
    return contact_matrix

In [ ]:
# generate input case

In [68]:
#get chr_list
chr_size_dict=get_chrom_size_dict('./Sus_scrofa.chrom.sizes')
chr_list=list(chr_size_dict.keys())

In [70]:
# get bin_index_list
a=0
bin_index_list=[1]
binsize="40000"
for chr_name in chr_list:
    chr_size=chr_size_dict[chr_name]
    bin_num=math.ceil(eval(chr_size)/eval(binsize))
    a+=bin_num
    bin_index_list.append(a+1)

In [79]:
#get contact_list
contact_list=read_contactfile("/home/zhengsq/SC_hic_pig/20220401/clean_results/hic_results/matrix/4c-15/raw/40000/4c-15_40000.matrix")

In [80]:
#Chromosome 1 input file generation
chr_name="1"#The chromosome name should be the same as the chr_list above
binsize="40000"

filename='./contact_matrix_chr'+chr_name+'.txt'
chr_contact_list=get_chr_contact(chr_name,contact_list,bin_index_list)
contact_matrix=get_contact_matrix(chr_contact_list,chr_name,binsize)
np.savetxt(filename,contact_matrix,delimiter="\t")

In [ ]:
#deDoc2
#deDoc2 can call TAD from both bulk cell and single cell data 
#/home/wanghy/software/deDoc2/deDoc2-main/deDoc2.jar
#github
#https://github.com/zengguangjie/deDoc2
#40k resolution command：
#java -jar deDoc2.jar -inputfile XXXX -binsize 40 -outputfile XXXX

In [56]:
import os
os.system("java -jar /home/wanghy/software/deDoc2/deDoc2-main/deDoc2.jar -inputfile contact_matrix_chr1.txt -binsize 40  -outputfile ./chr1")

reading input data...
predicting TLDs...


0